In [ ]:
##### Copyright 2020 The TensorFlow Authors.

#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Callback API 를 이용한 학습 흐름 제어

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/guide/keras/train_and_evaluate"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서 보기</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ko/guide/keras/train_and_evaluate.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ko/guide/keras/train_and_evaluate.ipynb">     <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">    GitHub에서 소스 보기</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ko/guide/keras/train_and_evaluate.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드</a></td>
</table>

## 시작하기

콜백은 훈련, 평가 또는 추론 중에 Keras 모델의 동작을 사용자 정의할 수 있는 강력한 도구입니다. TensorBoard로 훈련 진행 상황과 결과를 시각화하기 위한 `tf.keras.callbacks.TensorBoard` 또는 훈련 도중 모델을 주기적으로 저장하는 `tf.keras.callbacks.ModelCheckpoint` 등이 여기에 포함됩니다.

## 설정

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Preprocess the data (these are NumPy arrays)
x_train = x_train.reshape(60000, 784).astype("float32") / 255
x_test = x_test.reshape(10000, 784).astype("float32") / 255

y_train = y_train.astype("float32")
y_test = y_test.astype("float32")

# Reserve 10,000 samples for validation
x_val = x_train[-10000:]
y_val = y_train[-10000:]
x_train = x_train[:-10000]
y_train = y_train[:-10000]

나중에 재사용하기 위해 모델 정의 함수 생성

def get_compiled_model():

    inputs = keras.Input(shape=(784,), name="digits")
    x = layers.Dense(64, activation="relu", name="dense_1")(inputs)
    x = layers.Dense(64, activation="relu", name="dense_2")(x)
    outputs = layers.Dense(10, activation="softmax", name="predictions")(x)
    model = keras.Model(inputs=inputs, outputs=outputs)
    
    model.compile(
        optimizer="rmsprop",
        loss="sparse_categorical_crossentropy",
        metrics=["sparse_categorical_accuracy"],
    )
    return model


## 콜백 사용하기

Keras의 콜백은 훈련 중 다른 시점(epoch의 시작, 배치의 끝, epoch의 끝 등)에서 호출되며 다음과 같은 동작을 구현하는 데 사용할 수 있는 객체입니다.

- 훈련 중 서로 다른 시점에서 유효성 검사 수행(내장된 epoch당 유효성 검사에서 더욱 확장)
- 정기적으로 또는 특정 정확도 임계값을 초과할 때 모델 검사점 설정
- 훈련이 정체 된 것처럼 보일 때 모델의 학습 속도 변경
- 훈련이 정체 된 것처럼 보일 때 최상위 레이어의 미세 조정
- 교육이 종료되거나 특정 성능 임계 값을 초과 한 경우 전자 메일 또는 인스턴트 메시지 알림 보내기
- 기타

콜백은 `fit()` 에 대한 호출에 목록으로 전달 될 수 있습니다.

model = get_compiled_model()

callbacks = [
    keras.callbacks.EarlyStopping(   #오버피팅이 발생하기 전에 학습을 끝내기 위해 사용
        # Stop training when `val_loss` is no longer improving
        monitor="val_loss",
        # "no longer improving" being defined as "no better than 1e-2 less"
        min_delta=1e-2,
        # "no longer improving" being further defined as "for at least 2 epochs"
        patience=2,  #1번째까지 봐줌, 2번째 발생 시 stop 
        verbose=1,   
    )
]
model.fit(
    x_train,
    y_train,
    epochs=20,
    batch_size=64,
    callbacks=callbacks,
    validation_data = (x_val, y_val),
)

### 많은 내장 콜백을 사용할 수 있습니다

- `ModelCheckpoint` : 주기적으로 모델을 저장
- `EarlyStopping`: 훈련이 더 이상 유효성 검사 메트릭을 개선하지 못하는 경우 훈련을 중단
- `TensorBoard` : 학습 과정을 시각확 할 수 있는 TensorBoard에서 시각화 할수 있는 학습 이력 정보 생성
- `CSVLogger` : 손실 및 메트릭 데이터를 CSV 파일로 저장
- 기타

## ModelCheckpoint

상대적으로 큰 데이터세트에 대한 모델을 훈련시킬 때는 모델의 ModelCheckpoint 빈번하게 저장하는 것이 중요합니다.

이를 수행하는 가장 쉬운 방법은 `ModelCheckpoint` 콜백을 사용하는 것입니다.

del model

model = get_compiled_model()

callbacks = [
    keras.callbacks.ModelCheckpoint(   #특정 시점의 가중치만을 저장
        # Path where to save the model
        # The two parameters below mean that we will overwrite
        # the current checkpoint if and only if
        # the `val_loss` score has improved.
        # The saved model name will include the current epoch.
        filepath="mymodel_{epoch}",   #무슨 이름으로 저장할지 정함
        save_best_only=True,  # Only save a model if `val_loss` has improved.
        monitor="val_loss",
        verbose=1,
    )
]
model.fit(
    x_train, y_train, epochs=2, batch_size=64, callbacks=callbacks, validation_data= (x_val, y_val)
)

## 학습 속도 일정 사용하기

딥 러닝 모델을 훈련 할 때 일반적인 패턴은 훈련이 진행됨에 따라 점차적으로 학습을 줄이는 것입니다. 이것을 일반적으로 "학습률 감소"라고합니다.

learning rate 감소 스케줄은 정적(현재 에포크 또는 현재 배치 인덱스의 함수로서 미리 고정됨) 또는 동적(모델의 현재 행동, 특히 검증 손실에 대응) 일 수있다.

### 1). 옵티마이저로 schedule 전달하기

옵티 마이저에서 schedule 객체를 `learning_rate` 인수로 전달하여 정적 학습 속도 감소 스케줄을 쉽게 사용할 수 있습니다.

initial_learning_rate = 0.1
lr_schedule = keras.optimizers.schedules.ExponentialDecay(  # ExponentialDecay는 계속 같은 비율로 learning rate를 감소시켜준다.
    initial_learning_rate, decay_steps=100000, decay_rate=0.96, staircase=True
)

optimizer = keras.optimizers.RMSprop(learning_rate=lr_schedule)

model = get_compiled_model()

model.compile(
    optimizer=optimizer,
    loss="sparse_categorical_crossentropy",
    metrics=["sparse_categorical_accuracy"],
)

model.fit(
    x_train, y_train, epochs=4, batch_size=1, validation_split=0.2
)

`ExponentialDecay` , `PiecewiseConstantDecay` , `PolynomialDecay` 및 `InverseTimeDecay` 와 같은 몇 가지 기본 제공 일정을 사용할 수 있습니다.

### 2. 콜백을 사용하여 동적 learning rate schedule 구현

옵티마이저가 validation 데이터 메트릭에 액세스할 수 없으므로 동적 학습률 schedule(예: validation 데이터를 이용한 loss값이 더 이상 개선되지 않을 때 학습률 감소)을 구현 할 수 없습니다.

그러나 콜백은 validation 데이터 메트릭을 포함해 모든 메트릭에 액세스할 수 있습니다.

del model

model = get_compiled_model()

callbacks = [
    keras.callbacks.ReduceLROnPlateau(  #learning rate 감소, 특정한 값만큼 연속적으로 작업이 일어날때 사용
      monitor='val_loss',
      factor=0.1,    #learning rate 을 줄이기 위해 epoch마다 0.1만큼 곱해줌
      patience=2,
      cooldown=1,
      verbose=1
    )
]

model.fit(
    x_train, y_train, epochs=20, batch_size=64, callbacks=callbacks, validation_split=0.2
)

## 훈련 중 손실 및 메트릭 시각화하기

교육 중에 모델을 주시하는 가장 좋은 방법은 로컬에서 실행할 수있는 브라우저 기반 응용 프로그램 인 [TensorBoard](https://www.tensorflow.org/tensorboard) 를 사용하는 것입니다.

- 교육 및 평가를위한 손실 및 지표의 라이브 플롯
- (옵션) 레이어 활성화 히스토그램 시각화
- (옵션) `Embedding` 레이어에서 학습한 포함된 공간의 3D 시각화

pip와 함께 TensorFlow를 설치한 경우, 명령줄에서 TensorBoard를 시작할 수 있습니다.

```
tensorboard --logdir=/full_path_to_your_logs
```

### TensorBoard 콜백 사용하기

TensorBoard를 Keras 모델 및 fit 메서드와 함께 사용하는 가장 쉬운 방법은 `TensorBoard` 콜백입니다.

가장 간단한 경우로, 콜백에서 로그를 작성할 위치만 지정하면 바로 쓸 수 있습니다.

import datetime

# 학습데이터의 log를 저장할 폴더 생성 (지정)
log_dir = "./logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

%load_ext tensorboard

%tensorboard --logdir {log_dir}

del model

model = get_compiled_model()

callbacks = [
  keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
]

model.fit(
    x_train, y_train, epochs=20, batch_size=64, callbacks=callbacks, validation_split=0.2
)

# 자신만의 콜백 작성하기

## 콜백 메서드의 개요

### 전역 메서드

#### `on_(train|test|predict)_begin(self, logs=None)`

`fit`/`evaluate`/`predict` 시작 시 호출됩니다.

#### `on_(train|test|predict)_end(self, logs=None)`

`fit`/`evaluate`/`predict` 종료 시 호출됩니다.

### 훈련/테스트/예측을 위한 배치 레벨의 메서드

#### `on_(train|test|predict)_batch_begin(self, batch, logs=None)`

훈련/테스트/예측 중에 배치를 처리하기 직전에 호출됩니다.

#### `on_(train|test|predict)_batch_end(self, batch, logs=None)`

훈련/테스트/예측이 끝날 때 호출됩니다. 이 메서드에서 `logs`는 메트릭 결과를 포함하는 dict입니다.

### 에포크 레벨 메서드(훈련만 해당)

#### `on_epoch_begin(self, epoch, logs=None)`

훈련 중 epoch가 시작될 때 호출됩니다.

#### `on_epoch_end(self, epoch, logs=None)`

훈련 중 epoc가이 끝날 때 호출됩니다.

## 기본적인 예제

다음의 경우 로깅하는 간단한 사용자 정의 콜백을 정의합니다.

- `fit`/`evaluate`/`predict`가 시작하고 끝날 때
- 각 에포크가 시작하고 끝날 때
- 각 훈련 배치가 시작하고 끝날 때
- 각 평가(테스트) 배치가 시작하고 끝날 때
- 각 추론(예측) 배치가 시작하고 끝날 때

class myCallback(tf.keras.callbacks.Callback):     
  def on_epoch_end(self, epoch, logs={}):   #epoch이 끝날때마다 실행되게 함
       if logs.get('sparse_categorical_accuracy') > 0.6:
         print("blahblah")
         self.model.stop_training = True   #모델의 accuracy가 60퍼가 넘으면 학습 멈춤

del model

model = get_compiled_model()

callbacks = [
             myCallback()
]   #꼭 리스트에 담을 필요는 없음

model.fit(
    x_train, y_train, epochs=20, batch_size=64, callbacks=callbacks, validation_split=0.2
)

class CustomCallback(keras.callbacks.Callback):
    def on_train_begin(self, logs=None):
        keys = list(logs.keys())
        print("Starting training; got log keys: {}".format(keys))

    def on_train_end(self, logs=None):
        keys = list(logs.keys())
        print("Stop training; got log keys: {}".format(keys))

    def on_epoch_begin(self, epoch, logs=None):
        keys = list(logs.keys())
        print("Start epoch {} of training; got log keys: {}".format(epoch, keys))

    def on_epoch_end(self, epoch, logs=None):
        keys = list(logs.keys())
        print("End epoch {} of training; got log keys: {}".format(epoch, keys))

    def on_test_begin(self, logs=None):
        keys = list(logs.keys())
        print("Start testing; got log keys: {}".format(keys))

    def on_test_end(self, logs=None):
        keys = list(logs.keys())
        print("Stop testing; got log keys: {}".format(keys))

    def on_predict_begin(self, logs=None):
        keys = list(logs.keys())
        print("Start predicting; got log keys: {}".format(keys))

    def on_predict_end(self, logs=None):
        keys = list(logs.keys())
        print("Stop predicting; got log keys: {}".format(keys))

    def on_train_batch_begin(self, batch, logs=None):
        keys = list(logs.keys())
        print("...Training: start of batch {}; got log keys: {}".format(batch, keys))

    def on_train_batch_end(self, batch, logs=None):
        keys = list(logs.keys())
        print("...Training: end of batch {}; got log keys: {}".format(batch, keys))

    def on_test_batch_begin(self, batch, logs=None):
        keys = list(logs.keys())
        print("...Evaluating: start of batch {}; got log keys: {}".format(batch, keys))

    def on_test_batch_end(self, batch, logs=None):
        keys = list(logs.keys())
        print("...Evaluating: end of batch {}; got log keys: {}".format(batch, keys))

    def on_predict_batch_begin(self, batch, logs=None):
        keys = list(logs.keys())
        print("...Predicting: start of batch {}; got log keys: {}".format(batch, keys))

    def on_predict_batch_end(self, batch, logs=None):
        keys = list(logs.keys())
        print("...Predicting: end of batch {}; got log keys: {}".format(batch, keys))

del model

model = get_compiled_model()

model.fit(
    x_train,
    y_train,
    batch_size=128,
    epochs=1,
    verbose=0,
    validation_split=0.5,
    callbacks=[CustomCallback()],
)

res = model.evaluate(
    x_test, y_test, batch_size=128, verbose=0, callbacks=[CustomCallback()]
)

res = model.predict(x_test, batch_size=128, callbacks=[CustomCallback()])

### `logs` dict 사용법

`logs` dict에는 손실값과 배치 또는 에포크의 끝에 있는 모든 메트릭이 포함됩니다.

class LossAndErrorPrintingCallback(keras.callbacks.Callback):
    def on_train_batch_end(self, batch, logs=None):
        print("For batch {}, loss is {:7.2f}.".format(batch, logs["loss"]))

    def on_test_batch_end(self, batch, logs=None):
        print("For batch {}, loss is {:7.2f}.".format(batch, logs["loss"]))

    def on_epoch_end(self, epoch, logs=None):
        print(
            "The average loss for epoch {} is {:7.2f} "
            "and mean absolute error is {:7.2f}.".format(
                epoch, logs["loss"], logs["sparse_categorical_accuracy"]
            )
        )

del model

model = get_compiled_model()
model.fit(
    x_train,
    y_train,
    batch_size=128,
    epochs=2,
    verbose=0,
    callbacks=[LossAndErrorPrintingCallback()],
)

## Keras 콜백 애플리케이션의 예

### 최소 손실 시 조기 중지

이 첫 번째 예는 `self.model.stop_training` (boolean) 속성을 설정하여 최소 손실에 도달했을 때 훈련을 중단하는 `Callback`을 생성하는 방법을 보여줍니다. 선택적으로, 로컬 최소값에 도달한 후 중단하기 전에 기다려야 하는 에포크 수를 지정하는 인수 `patience`을 제공할 수 있습니다.

`tf.keras.callbacks.EarlyStopping`은 더 완전한 일반적인 구현을 제공합니다.

import numpy as np


class EarlyStoppingAtMinLoss(keras.callbacks.Callback):   #Callback 클래스 상속받음
    """Stop training when the loss is at its min, i.e. the loss stops decreasing.

  Arguments:
      patience: Number of epochs to wait after min has been hit. After this
      number of no improvement, training stops.
  """

    def __init__(self, patience=0):     #initialize(초기화) 함수
        super(EarlyStoppingAtMinLoss, self).__init__()   #부모클래스에 접근(super), 부모 클래스 초기화
        self.patience = patience   #멤버변수로 등록
        # best_weights to store the weights at which the minimum loss occurs.
        self.best_weights = None   #최고의 가중치 값들을 저장하기 위한 멤버 변수

    def on_train_begin(self, logs=None):   #training 시작때 호출됨
        # The number of epoch it has waited when loss is no longer minimum.
        self.wait = 0   #몇 번 patient 가 일어났는지 저장
        # The epoch the training stops at.
        self.stopped_epoch = 0
        # Initialize the best as infinity.
        self.best = np.Inf     #무한대값으로 초기화

    def on_epoch_end(self, epoch, logs=None):    #epoch이 끝날때 호출됨
        current = logs.get("loss")   
        if np.less(current, self.best):  #무한대값은 연산자로 비교 불가, less함수 사용
            self.best = current #current가 best보다 작으므로, best를 current로 바꿔줌
            self.wait = 0  
            # Record the best weights if current results is better (less).
            self.best_weights = self.model.get_weights()   #모델에 접근해서 get_weights함수를 이용해서 최고의 weight를 업데이트함
        else:  #반대 경우--> 성능이 나빠지는 경우 학습을 중단시키는 작업
            self.wait += 1     
            if self.wait >= self.patience:   #patience만큼  wait가 올라가게 되면 중단을 시키는 것
            #최소 손실 시 중지 코드에서 patience는 총 횟수가 아니라 연속으로 나온 횟수

  
                self.stopped_epoch = epoch
                self.model.stop_training = True   #멈춰! 
                print("Stop Training")
                self.model.set_weights(self.best_weights)  #멈친 상태의 모델의 가중치는 베스트가 아니기 때문에 기존에 저장해둔 best_weights를 모델의 weight로 설정  

    def on_train_end(self, logs=None):   #학습이 끝날 때 epoch이 끝났음을 알려주는 문구 출력
        if self.stopped_epoch > 0:
            print("Epochs : [] training stopped" .format(self.stopped_epoch))

del model

model = get_compiled_model()

model.fit(
    x_train,
    y_train,
    batch_size=64,
    steps_per_epoch=5,
    epochs=30,
    verbose=0,
    callbacks=[LossAndErrorPrintingCallback(), EarlyStoppingAtMinLoss(2)],
)

### 학습 속도 스케줄링

이 예제에서는 사용자 정의 콜백을 사용하여 훈련 동안 옵티마이저의 학습 속도를 동적으로 변경하는 방법을 보여줍니다.

보다 일반적인 구현에 대해서는 `callbacks.LearningRateScheduler`를 참조하세요.

class CustomLearningRateScheduler(keras.callbacks.Callback):
    """Learning rate scheduler which sets the learning rate according to schedule.

  Arguments:
      schedule: a function that takes an epoch index
          (integer, indexed from 0) and current learning rate
          as inputs and returns a new learning rate as output (float).
  """

    def __init__(self, schedule):   
        super(CustomLearningRateScheduler, self).__init__()  #부모클래스 불러와서 초기화
        self.schedule = schedule    #전달받은 schedule함수를 멤버 변수로(함수형...생소하네ㅠ) 

    def on_epoch_begin(self, epoch, logs=None):  #learning rate 떨어뜨리는 작업
        if not hasattr(self.model.optimizer, "lr"):  
            raise ValueError('Optimizer must have a "lr" attribute.')
        # Get the current learning rate from model's optimizer.
        lr = float(tf.keras.backend.get_value(self.model.optimizer.learning_rate))  #모델에 있는 learning rate값을 가져옴 tf.keras.backend.get_value()
        # Call schedule function to get the scheduled learning rate.
        scheduled_lr = self.schedule(epoch, lr)
        # Set the value back to the optimizer before this epoch starts
        tf.keras.backend.set_value(self.model.optimizer.lr, scheduled_lr)
        print("\nEpoch %05d: Learning rate is %6.4f." % (epoch, scheduled_lr))  #잘 변경되었는지 확인하는 log 찍기


LR_SCHEDULE = [
    # (epoch to start, learning rate) tuples
    (3, 0.05),
    (6, 0.01),
    (9, 0.005),
    (12, 0.001),
]


def lr_schedule(epoch, lr):  
    """Helper function to retrieve the scheduled learning rate based on epoch."""
    if epoch < LR_SCHEDULE[0][0] or epoch > LR_SCHEDULE[-1][0]:
        return lr
    for i in range(len(LR_SCHEDULE)):
        if epoch == LR_SCHEDULE[i][0]:
            return LR_SCHEDULE[i][1]
    return lr

del model

model = get_compiled_model()
model.fit(
    x_train,
    y_train,
    batch_size=64,
    steps_per_epoch=5,
    epochs=15,
    verbose=0,
    callbacks=[
        LossAndErrorPrintingCallback(),
        CustomLearningRateScheduler(lr_schedule),
    ],
)